# Jupyter to translate csv from darija to english

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00


In [4]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=ed2062b4074770ede09f80f946a98d9f07c5fb3aeafe42f94f060032ea24200b
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 4.0.0rc1
    Uninstalling googletrans-4.0.0rc1:
      Successfully uninstalled googletrans-4.0.0rc1


In [7]:
import torch
from transformers import NllbTokenizerFast, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd

'''
class to use nllb
'''
class Darija2Eng :
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(
        "facebook/nllb-200-distilled-600M",
        )

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to(self.device)


    def prediction(self, article :str):
        inputs = self.tokenizer(article, return_tensors="pt").to(self.device)

        translated_tokens = self.model.generate(
            **inputs, forced_bos_token_id=self.tokenizer.lang_code_to_id["eng_Latn"], max_length=30
        )
        outputs = self.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

        return outputs

In [8]:
# @st.cache_resource
def parse_gg_sheet(url):
    print("Parsing Google Sheet:", url)
    url = url.replace("edit#gid=", "export?format=csv&gid=")
    df = pd.read_csv(url, on_bad_lines="warn")
    return df

In [54]:
def translate_df(df, columns_to_translate, prediction_func, file_name):
  # iteration on the columns
  df[columns_to_translate] = df[columns_to_translate].fillna(value="").copy()
  for col in columns_to_translate:
      col_trad = prediction_func(col).text
      if col_trad==col :
        col_trad = f"trad_{col_trad}"
      print(col_trad)
      try :
        df[f"{col_trad}"] = df[col].apply(translator.translate, dest='en').apply(getattr, args=('text',))
      except Exception as e:
        try  :
          df[f"{col_trad}"] = df[col].apply(prediction_func)
        except Exception as err:
          print(err)
  df.to_csv(f"{file_name}.csv", index = False)

In [10]:
import pandas as pd
from typing import Union


VERIFIED_REQUESTS_URL = (
    "https://docs.google.com/spreadsheets/d/1PXcAtI5L95hHSXAiRl3Y4v5O4coG39S86OTfBEcvLTE/edit#gid=0"
)
REQUESTS_URL = "https://docs.google.com/spreadsheets/d/1gYoBBiBo1L18IVakHkf3t1fOGvHWb23loadyFZUeHJs/edit#gid=966953708"
INTERVENTIONS_URL = (
    "https://docs.google.com/spreadsheets/d/1eXOTqunOWWP8FRdENPs4cU9ulISm4XZWYJJNR1-SrwY/edit#gid=2089222765"
)

In [18]:
from googletrans import Translator
translator = Translator()

In [67]:
darija2eng_obj = Darija2Eng()

## Assessing the model quality on some real values from the forms

In [78]:
sentences = ["Rabate d'oyore  kaye bato f zan9a",
              "Dowar ait l3arbi sdi hmade oulade barhile",
              'مكن تعيطي ليها في لبورطابل',
              'مأوى',
              'إغاثة',
              'ساعدة طبية']

### nllb 600
advantage : can translate darija
disadvantage :
- takes time to compute (code can be optimized)
- no latine letters translation possible
- no always very accurate

In [85]:
nllb_trad = []
for sentence in sentences:
  nllb_trad.append(darija2eng_obj.prediction(article = sentence))

In [90]:
nllb_trad

['The Commission shall adopt the following measures:',
 'Dowar ait l3arbi sdi hme made or barhile',
 "I can't wait to get her in the hospital.",
 'The following is a list of the countries of the European Union:',
 'The following is the list of the categories:',
 'Help me to get better.']

### google translate
advantage :
- can translate darija
- fast

disadvantage :
- no latine letters translation possible
- no always very accurate

In [87]:
translations = translator.translate(["Rabate d'oyore  kaye bato f zan9a",
                                     "Dowar ait l3arbi sdi hmade oulade barhile",
                                     'مكن تعيطي ليها في لبورطابل',
                                     'مأوى',
                                     'إغاثة',
                                     'ساعدة طبية']
                                    )

In [88]:
google_trans = []
for translation in translations :
  google_trans.append(translation.text)

In [89]:
google_trans

['Link to the place where you are in a street',
 'The role of a verse by Arabi Sadi Hamad Awlad Barheel',
 'Can you give it to her in Portal?',
 'Shelter',
 'Relief',
 'Medical assistance']

### comparision summary

In [96]:
pd.DataFrame(list(zip(sentences, nllb_trad, google_trans)), columns=['orginal_text', 'nllb','google_trad'])

,orginal_text,nllb,google_trad
0,Rabate d'oyore kaye bato f zan9a,The Commission shall adopt the following measu...,Link to the place where you are in a street
1,Dowar ait l3arbi sdi hmade oulade barhile,Dowar ait l3arbi sdi hme made or barhile,The role of a verse by Arabi Sadi Hamad Awlad ...
2,مكن تعيطي ليها في لبورطابل,I can't wait to get her in the hospital.,Can you give it to her in Portal?
3,مأوى,The following is a list of the countries of th...,Shelter
4,إغاثة,The following is the list of the categories:,Relief
5,ساعدة طبية,Help me to get better.,Medical assistance


## Traduction on request form

In [63]:
df = parse_gg_sheet(REQUESTS_URL)

Parsing Google Sheet: https://docs.google.com/spreadsheets/d/1gYoBBiBo1L18IVakHkf3t1fOGvHWb23loadyFZUeHJs/edit#gid=966953708


In [64]:
df.columns

Index(['Horodateur', 'ما هي احتياجاتك؟ (أضفها إذا لم يتم ذكرها)',
       'ما هي أقرب مدينة إليك؟ (أضفها إذا لم يتم ذكرها)',
       '  لأي  جماعة / قيادة / دوار تنتمون ؟',
       'هل يمكنك تقديم الإحداثيات الدقيقة للموقع؟ (ادا كنت لا توجد بعين المكان) متلاً \n31.01837503440344, -6.781405948842175',
       'الرجاء الضغط على الرابط التالي لمعرفة موقعك إذا كان متاحا',
       'هل توجد بعين المكان؟', 'العنوان أو اسم القرية (عند الامكان)',
       '(عند الامكان) رقم هاتف شخص موجود في عين المكان',
       '(إختياري) هل يمكنك أن تعطينا المزيد من المعلومات حول الوضع؟',
       'مصدر المعلومات (إختياري)', 'Status', 'GeoStamp', 'GeoCode',
       'GeoAddress',
       'هل يمكنك تقديم الإحداثيات الدقيقة للموقع؟ (ادا كنت لا توجد بعين المكان) متلاً \n31.01837503440344, -6.781405948842175.1',
       'Verified', 'GeoStamp.1', 'GeoCode.1', 'GeoAddress.1', 'GeoStamp.2',
       'GeoCode.2', 'GeoAddress.2'],
      dtype='object')

In [65]:
requests_columns_to_translate = ['ما هي احتياجاتك؟ (أضفها إذا لم يتم ذكرها)',
       'ما هي أقرب مدينة إليك؟ (أضفها إذا لم يتم ذكرها)',
       '  لأي  جماعة / قيادة / دوار تنتمون ؟',
       'هل يمكنك تقديم الإحداثيات الدقيقة للموقع؟ (ادا كنت لا توجد بعين المكان) متلاً \n31.01837503440344, -6.781405948842175',
       'الرجاء الضغط على الرابط التالي لمعرفة موقعك إذا كان متاحا',
       'هل توجد بعين المكان؟'
        , 'العنوان أو اسم القرية (عند الامكان)',
       '(إختياري) هل يمكنك أن تعطينا المزيد من المعلومات حول الوضع؟',
       'مصدر المعلومات (إختياري)']


In [66]:
%%time
translate_df(df=df, columns_to_translate=requests_columns_to_translate, prediction_func=translator.translate, file_name="form_requests_trad")

What are your needs? (add if not mentioned)
What is the closest city to you? (add if not mentioned)
To which group / leadership / role do you belong?
Can you provide the exact coordinates of the location? (If you are not there in the place) like this
31.01837503440344, -6.781405948842175
Please click on the following link to see your location if available
Is it located on site?
Address or village name (if possible)
(Optional) Can you give us more information about the situation?
Source of information (optional)
CPU times: user 1min 58s, sys: 7.8 s, total: 2min 5s
Wall time: 9min 48s


## Traduction of verified requests

In [51]:
verified_df = parse_gg_sheet(VERIFIED_REQUESTS_URL)
verified_df.columns

Parsing Google Sheet: https://docs.google.com/spreadsheets/d/1PXcAtI5L95hHSXAiRl3Y4v5O4coG39S86OTfBEcvLTE/edit#gid=0


Index(['VerificationStatus', 'Verification Date', 'Help Details',
       'Further details', 'Phone Number', 'Location Details',
       'Emergency Degree', 'Location Link/GPS Coordinates', 'Status',
       'Intervenant ', 'Intervention Date', 'Any remarks',
       'Automatic Extracted Coordinates'],
      dtype='object')

In [52]:
verified_columns_to_translate = ["Help Details", "Further details", "Location Details"]

In [55]:
%%time
translate_df(df=verified_df, columns_to_translate=verified_columns_to_translate, prediction_func=translator.translate, file_name="form_verif_requests_trad")

trad_Help Details
trad_Further details
trad_Location Details
CPU times: user 6.04 s, sys: 379 ms, total: 6.42 s
Wall time: 29.8 s


## Traduction of verified requests

In [56]:
interventions_df = parse_gg_sheet(INTERVENTIONS_URL)
interventions_df.columns

Parsing Google Sheet: https://docs.google.com/spreadsheets/d/1eXOTqunOWWP8FRdENPs4cU9ulISm4XZWYJJNR1-SrwY/edit#gid=2089222765


Index(['Horodateur', 'Nom de l'organisme  | Organization Name',
       'Informations de Contact | Contact Information',
       'Lien Google Maps | Google Maps Link',
       'Date de l'intervention | Intervention date',
       'Pouvez-vous nous préciser si vous êtes déjà intervenus ou si vous prévoyez de le faire | Tell us if you already made the intervention, or if you're planning to do it',
       'Types d'assistance fournie | Types of Assistance Provided',
       'Etat de la région actuel | Current situation of the area ',
       'Si possible, pouvez-vous fournir plus de détails sur l'intervention ? | If possible, Can you provide more details about the intervention? ',
       'Nom du Douar ou Caïdat | Douar or Caïdat name ',
       'Toute information complémentaire est la bienvenue / Any Additional info is welcome',
       'Population du douar si disponible | Population of the douar if available',
       'Etat de La route au douar | State of the road to the douar',
       'Automatic 

In [57]:
interventions_columns_to_translate = ["Pouvez-vous nous préciser si vous êtes déjà intervenus ou si vous prévoyez de le faire | Tell us if you already made the intervention, or if you're planning to do it",
       "Types d'assistance fournie | Types of Assistance Provided",
       "Etat de la région actuel | Current situation of the area ",
       "Si possible, pouvez-vous fournir plus de détails sur l'intervention ? | If possible, Can you provide more details about the intervention? ",
       "Nom du Douar ou Caïdat | Douar or Caïdat name ",
       "Toute information complémentaire est la bienvenue / Any Additional info is welcome",
       "Population du douar si disponible | Population of the douar if available",
       "Etat de La route au douar | State of the road to the douar"]

In [62]:
%%time
translate_df(df=interventions_df, columns_to_translate=interventions_columns_to_translate, prediction_func=translator.translate, file_name="form_inteventions_trad")

Can you tell us if you have already intervened or if you plan to do so | Tell us if you already made the intervention, or if you're planning to do it
trad_Types d'assistance fournie | Types of Assistance Provided
Current state of the region | Current situation of the area
If possible, can you provide more details about the intervention? | If possible, Can you provide more details about the intervention?
Name of Douar or Caïdat | Douar or Caïdat name
Any additional info is welcome
Douar population if available | Population of the douar if available
trad_Etat de La route au douar | State of the road to the douar
CPU times: user 11.9 s, sys: 819 ms, total: 12.7 s
Wall time: 1min 2s
